## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Mobile

### Links: 
___

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.sessions_st_mobile
    (
        `report_date` Date,
        `citizen_id`  Int32,
        `created_at` DateTime,
        `last_use` DateTime,
        `updated_at` DateTime,
        `platform` String,
        `call_enabled` Int16,
        `app` String,
        `logged_in` Int16,
        `timezone` String
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/sessions_st_mobile/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [16]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.sessions_st_mobile_ch
    (
        `report_date` Date,
        `citizen_id`  Int32,
        `created_at` DateTime,
        `last_use` DateTime,
        `updated_at` DateTime,
        `platform` String,
        `call_enabled` Int16,
        `app` String,
        `logged_in` Int16,
        `timezone` String
    )
    ENGINE = MergeTree()
    ORDER BY citizen_id
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [17]:
# creating a materialized view

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.sessions_st_mobile_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.sessions_st_mobile_ch AS
    SELECT
        *
    FROM db1.sessions_st_mobile
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

____


In [2]:
# creating a materialized view

query_text = """--sql
    SELECT
        *
    FROM db1.sessions_st_mobile_ch
    ORDER BY report_date DESC
    LIMIT 10
    """

ch.query_run(query_text)

,report_date,citizen_id,created_at,last_use,updated_at,platform,call_enabled,app,logged_in,timezone
0,2025-09-14,6957,2019-11-30 12:54:33,2025-09-14 10:17:56,2025-09-14 10:17:56,android,1,default,1,Asia/Krasnoyarsk
1,2025-09-14,6964,2019-11-30 14:19:00,2025-09-14 15:42:23,2025-09-14 15:42:23,android,0,default,1,Asia/Krasnoyarsk
2,2025-09-14,6979,2019-11-30 16:00:29,2025-09-14 18:06:26,2025-09-14 18:06:26,ios,1,default,1,Asia/Irkutsk
3,2025-09-14,7142,2023-03-30 12:18:45,2025-09-14 15:39:03,2025-09-14 15:39:03,ios,1,default,1,Europe/Samara
4,2025-09-14,7017,2024-07-27 08:49:15,2025-09-14 14:34:40,2025-09-14 14:34:40,hms,0,default,1,Asia/Irkutsk
5,2025-09-14,7019,2023-12-09 15:15:56,2025-09-14 10:48:50,2025-09-14 10:48:50,hms,1,default,1,Asia/Irkutsk
6,2025-09-14,7032,2019-12-01 13:45:57,2025-09-14 18:11:38,2025-09-14 18:11:38,ios,0,default,1,Europe/Moscow
7,2025-09-14,7009,2019-12-01 04:30:49,2025-09-14 07:20:17,2025-09-14 07:20:17,android,1,default,1,Asia/Irkutsk
8,2025-09-14,7100,2019-12-02 11:16:05,2025-09-14 16:38:58,2025-09-14 16:38:58,ios,1,default,1,Asia/Irkutsk
9,2025-09-14,7041,2019-12-01 15:02:52,2025-09-14 14:18:24,2025-09-14 14:18:24,ios,1,default,1,Asia/Irkutsk


In [12]:
query_text = """--sql
SELECT
	*
FROM db1.`sessions_st_mobile`
WHERE _path LIKE '%/year=2025/month=7/7%' 
   OR _path LIKE '%/year=2025/month=07/07%'
ORDER BY report_date DESC
LIMIT 10
"""

ch.query_run(query_text)

report_date,citizen_id,created_at,last_use,updated_at,platform,call_enabled,app,logged_in,timezone
date,i64,datetime[ns],datetime[ns],datetime[ns],str,i64,str,i64,str
2025-07-07,593190,2022-03-31 22:15:32,2025-07-08 00:15:40,2025-07-08 00:15:40,"""android""",1,"""default""",1,"""Europe/Kaliningrad"""
2025-07-07,593169,2022-03-31 21:54:52,2025-07-07 18:21:22,2025-07-07 18:21:22,"""ios""",0,"""default""",1,"""Europe/Moscow"""
2025-07-07,593161,2022-03-31 21:49:13,2025-07-08 00:04:20,2025-07-08 00:04:20,"""android""",1,"""default""",1,"""Europe/Moscow"""
2025-07-07,593113,2022-03-31 21:10:45,2025-07-07 20:43:54,2025-07-07 20:43:54,"""android""",1,"""default""",1,"""Asia/Krasnoyarsk"""
2025-07-07,593107,2022-03-31 21:03:02,2025-07-07 17:29:11,2025-07-07 17:29:11,"""ios""",1,"""default""",1,"""Asia/Novokuznetsk"""
2025-07-07,593096,2022-03-31 21:02:26,2025-07-07 23:39:51,2025-07-07 23:39:51,"""android""",1,"""default""",1,"""Europe/Moscow"""
2025-07-07,593001,2022-03-31 19:59:18,2025-07-07 22:16:34,2025-07-07 22:16:34,"""android""",1,"""default""",1,"""Europe/Moscow"""
2025-07-07,593092,2022-03-31 20:51:38,2025-07-07 11:17:37,2025-07-07 11:17:37,"""ios""",1,"""default""",1,"""Europe/Samara"""
2025-07-07,593098,2022-03-31 20:55:18,2025-07-07 21:09:36,2025-07-07 21:09:36,"""ios""",1,"""default""",1,"""Europe/Moscow"""


___
## Drop the Table

In [14]:
query_text = """
    DROP TABLE db1.sessions_st_mobile_ch
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
1781355,"""438cbbba-80de-481f-8b28-b569fd…"


In [3]:
query_text = """
    DROP TABLE db1.sessions_st_mobile_mv
    """
ch.query_run(query_text)

,read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
0,0,0,0,0,0,0,0,2522307,fc822f82-56e6-4214-a8f0-f4113adaaf50


___

## Refreshing the data

In [ ]:

query_text = """
SYSTEM REFRESH VIEW db1.sessions_st_mobile_mv
"""

ch.query_run(query_text)